In [1]:
import numpy as np
import os
import glob
import cv2

from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPool2D, Activation,Dropout
from keras.models import Model
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


# Model
Below are different model functions 

In [2]:
# model with ALL layers frozen, except for the SOFTNAX layer
def get_model_exp1_a():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax')(X)
    
    # magical line of freezing layers
    for layer in base_model.layers:
        layer.trainable=False
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model=get_model_exp1_a()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [3]:
# model with all layers UNFROZEN, GPU might explode!
def get_model_exp1_b():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax')(X)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model=get_model_exp1_b()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [4]:
# model with only the SOFTMAX layer frozen, unconventional wisdom kicks in!
def get_model_exp2_a():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax',trainable=False)(X)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model=get_model_exp2_a()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [5]:
# model with SOFTMAX layer and last 4 vgg layers frozen, others layers remain unfrozen unconventional wisdom for the win!
def get_model_exp2_b():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax',trainable=False)(X)
    
    for layer in base_model.layers[-4:]:
        layer.trainable=False
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model=get_model_exp2_b()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [6]:
def get_model_exp2_c():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    
    # last layer unfrozen here
    predictions = Dense(10, activation='softmax',trainable=True)(X)
    
    # intermediate layers fronzen
    for layer in base_model.layers[-5:]:
        layer.trainable=False
    #vgg divided in two parts, trained jointly
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

model=get_model_exp2_c()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [7]:
# model with only 1/0 business
def get_model_exp2_d():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax')(X)
    
    #for layer in base_model.layers[::2]:
    #        layer.trainable = False
    
    model = Model(inputs=base_model.input, outputs=predictions)
    weights = model.layers[-1].get_weights()
   
    # do stuff here
    w1 = np.zeros( weights[0].shape )
    w2 = np.zeros( weights[1].shape )
    w  = [w1,w2]
    for i in range(0,10):
        w1[i*50:(i+1)*50,i] = 1.0
    print(w1.shape)
    model.layers[-1].set_weights( w)
    print(w2)
    model.layers[-1].trainable=False
    
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model=get_model_exp2_d()
model.summary()

(512, 10)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None

In [8]:
print("Done!")

Done!


In [ ]:
# Dummy code for training loop
# Trained like this in numta
path_model='' 

K.set_value(model.optimizer.lr,1e-2) # set the learning rate

h=model.fit(x=x_train,     
            y=y_train, 
            batch_size=128, 
            epochs=100, 
            verbose=1, 
            validation_data=(X_val,y_val_aug),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )

